## 케라스 콜백 사용 준비하기

In [ ]:
from tensorflow.keras.datasets import mnist

# 텐서플로우 저장소에서 데이터를 다운로드 받습니다.
(x_train, y_train), (x_test, y_test) = mnist.load_data(path = 'mnist.npz')

from sklearn.model_selection import train_test_split

# 훈련/검증 데이터를 얻기 위해 0.7/0.3의 비율로 분리합니다.
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.3, random_state = 777)

num_x_train = x_train.shape[0]
num_x_val = x_val.shape[0]
num_x_test = x_test.shape[0]

# 모델의 입력으로 사용하기 위한 전처리 과정입니다.
x_train = (x_train.reshape(-1, 28, 28, 1)) / 255
x_val = (x_val.reshape(-1, 28, 28, 1)) / 255
x_test = (x_test.reshape(-1, 28, 28, 1)) / 255

from tensorflow.keras.utils import to_categorical

# 각 데이터의 레이블을 범주형 형태로 변경합니다.
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers import Input

# 함수형 API는 Input()을 통해 입력값의 형태를 정의해주어야 합니다.
inputs = Input(shape = (28, 28, 1))
x = Conv2D(32, (3, 3), activation = 'relu')(inputs)
x = Conv2D(32, (3, 3), activation = 'relu')(x)
x = MaxPooling2D(strides = 2)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(10, activation = 'softmax')(x)

# 위에서 정의한 층을 포함하고 있는 모델을 생성합니다.
model = Model(inputs = inputs, outputs = x)

model.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy',
              metrics = ['acc'])

## ModelCheckpoint 콜백 사용하기

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = './best_model.hdf5'

# 콜백을 정의합니다.
callbacks = [ModelCheckpoint(filepath = filepath, monitor = 'val_loss', verbose = 1,
                           save_best_only = True)]

# callbacks 인자를 통해 정의한 콜백을 전달합니다.
model.fit(x_train, y_train,
         batch_size = 32,
         validation_data = (x_val, y_val),
         epochs = 10,
         callbacks = callbacks)

## EarlyStopping 콜백 사용하기

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# 콜백을 정의합니다.
callbacks = [EarlyStopping(monitor = 'val_loss', patience = 3, verbose = 1)]

# callbacks 인자를 통해 정의한 콜백을 전달합니다.
model.fit(x_train, y_train,
         batch_size = 32,
         validation_data = (x_val, y_val),
         epochs = 30,
         callbacks = callbacks)

## ReduceLROnPlataeu 콜백 사용하기

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# 콜백을 정의합니다.
callbacks = [ReduceLROnPlateau(monitor = 'val_loss', patience = 3, factor = 0.2, 
                               verbose = 1, min_lr = 1e-5)]

# callbacks 인자를 통해 정의한 콜백을 전달합니다.
model.fit(x_train, y_train,
         batch_size = 32,
         validation_data = (x_val, y_val),
         epochs = 50,
         callbacks = callbacks)

## TensorBoard 콜백 사용하기

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

logdir = './logs'

# 콜백을 정의합니다.
callbacks = [TensorBoard(log_dir = logdir, histogram_freq = 1, 
                        write_graph = True, write_images = True)]

# callbacks 인자를 통해 정의한 콜백을 전달합니다.
model.fit(x_train, y_train,
         batch_size = 32,
         validation_data = (x_val, y_val),
         epochs = 30,
         callbacks = callbacks)

## Tensorboard 실행

In [ ]:
!tensorboard dev upload --logdir ./logs/ --name "My test" --description "This is my first tensorboard"